# Analyse BlueSeis BSPF Events

With pressure sensor parascientific and new sensor 

In [1]:
import os 
import obspy as obs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# from andbro__savefig import __savefig
from obspy.clients.fdsn import Client

## Configurations

In [2]:
if os.uname().nodename == 'lighthouse':
    root_path = '/home/andbro/'
    data_path = '/home/andbro/kilauea-data/'
    archive_path = '/home/andbro/freenas/'
elif os.uname().nodename == 'kilauea':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'

In [3]:
config = {}

config['minlatitude'] = 31 -1
config['maxlatitude'] = 35 +1
config['minlongitude'] = -119 -1
config['maxlongitude'] = -114 +1

config['BSPF_lon'] = np.array([-116.455439])
config['BSPF_lat'] = np.array([33.610643])

config['minmagnitude'] = 2.5

config['tbeg'] = obs.UTCDateTime("2022-10-01")
config['tend'] = obs.UTCDateTime("2023-03-31")

config['eventfile'] = "BSPF_event_catalog_extended"

config['trigger'] = data_path+"/BSPF/trigger/"

config['outpath'] = data_path+"/BSPF/data/"

In [4]:
def __export_new_events(config, events_old, events):
    
    ## combine new and old catalog
    tmp = pd.concat([__cat_to_df(events_old), __cat_to_df(events)]).reset_index(drop=False)
    ## remove duplicates
    df = tmp.drop_duplicates(subset=['index'], keep=False)
    ## sort and set index
    df = df.sort_index(ascending=False)
    df.set_index('index', inplace=True, drop=True)
    
    ## export new events to pickle file
    print(f" -> export new events: {config['outpath']}new_events.pkl")
    df.to_pickle(config['outpath']+"new_events.pkl")
    
    del tmp, df  

In [5]:
def __cat_to_df(cat):
    
    from pandas import DataFrame
    
    times = []
    lats = []
    lons = []
    deps = []
    magnitudes = []
    magnitudestype = []
    
    for event in cat:
        if len(event.origins) != 0 and len(event.magnitudes) != 0:
            times.append(event.origins[0].time.datetime)
            lats.append(event.origins[0].latitude)
            lons.append(event.origins[0].longitude)
            deps.append(event.origins[0].depth)
            magnitudes.append(event.magnitudes[0].mag)
            magnitudestype.append(event.magnitudes[0].magnitude_type )
            
    df = DataFrame({'latitude':lats,'longitude':lons,'depth':deps,
                    'magnitude':magnitudes,'type':magnitudestype}, 
                     index = times
                  )
    
    return df

In [6]:
def __add_distances_and_backazimuth(config, df):

    from obspy.geodetics.base import gps2dist_azimuth

    dist = np.zeros(len(df))
    baz = np.zeros(len(df))

    
    for ii, ev in enumerate(df.index):
        try:
            dist[ii], az, baz[ii] = gps2dist_azimuth(config['BSPF_lat'], config['BSPF_lon'],
                                                     df.latitude[ii], df.longitude[ii],
                                                     a=6378137.0, f=0.0033528106647474805
                                                     )
        except:
            print(" -> failed to compute!")
            
    df['backazimuth'] = baz
    df['distances_km'] = dist/1000

    return df

## Get Events

In [7]:
client = Client("USGS")

## events - all in area and time period
events_all = client.get_events(minlatitude=config['minlatitude'], maxlatitude=config['maxlatitude'],
                               minlongitude=config['minlongitude'], maxlongitude=config['maxlongitude'],
                               starttime=config['tbeg'],
                               endtime=config['tend'],
                               )

# ## events smaller than 2.0
# events_1 = client.get_events(minlatitude=config['minlatitude'], maxlatitude=config['maxlatitude'],
#                              minlongitude=config['minlongitude'], maxlongitude=config['maxlongitude'],
#                              starttime=config['tbeg'],
#                              endtime=config['tend'],
#                              maxmagnitude=2.0,
#                             )

## events between 2.0 and 3.0 within distance 0.5 degrees
events_2 = client.get_events(
                             latitude=config['BSPF_lat'], longitude=config['BSPF_lon'],
                             starttime=config['tbeg'],
                             endtime=config['tend'],
                             minmagnitude=2.0,
                             maxmagnitude=3.0,
                             maxradius=0.5,
                            )

## events between 2.0 and 3.0 within distance 2.0 degrees
events_3 = client.get_events(
                             latitude=config['BSPF_lat'], longitude=config['BSPF_lon'],
                             starttime=config['tbeg'],
                             endtime=config['tend'],
                             minmagnitude=3.0,
                             maxmagnitude=5.0,
                             maxradius=5.0,
                            )

## events larger than 5.0 
events_4 = events_all.filter("magnitude > 5.0")


## join specified event catalogs together
events = events_2 + events_3 + events_4


# events.plot(projection="local");

events.write(config['outpath']+config['eventfile']+".xml", format="QUAKEML")
events_all.write(config['outpath']+config['eventfile']+"_all.xml", format="QUAKEML")

/home/andbro/anaconda3/envs/pygmt/lib/python3.9/site-packages/obspy/io/quakeml/core.py:1112: UserWarning: 'quakeml:earthquake.usgs.gov/fdsnws/event/1/query?starttime=2022-10-01T00%3A00%3A00.000000&endtime=2023-03-31T00%3A00%3A00.000000&minlatitude=30.0&maxlatitude=36.0&minlongitude=-120.0&maxlongitude=-113.0' is not a valid QuakeML URI. It will be in the final file but note that the file will not be a valid QuakeML file.
  warnings.warn(msg % obj.id)


In [8]:
## convert catalog object to data frame
events_df = __cat_to_df(events)
events_all_df = __cat_to_df(events_all)

## add epicentral distances
__add_distances_and_backazimuth(config, events_df)
__add_distances_and_backazimuth(config, events_all_df)

## write data frame as pickle file
events_df.to_pickle(config['outpath']+config['eventfile']+".pkl")
events_all_df.to_pickle(config['outpath']+config['eventfile']+"_all.pkl")

## Plot Event Timeline

In [9]:
events_all_df = events_all_df.rename_axis('timestamp').reset_index()
events_all_df['seconds'] = [abs((events_all_df.timestamp.min()-tt).total_seconds()) for tt in events_all_df.timestamp]

events_df = events_df.rename_axis('timestamp').reset_index()
events_df['seconds'] = [abs((events_df.timestamp.min()-tt).total_seconds()) for tt in events_df.timestamp]
events_df

,timestamp,latitude,longitude,depth,magnitude,type,backazimuth,distances_km,seconds
0,2023-03-27 09:28:56.340,33.785667,-116.053500,7370.0,2.58,ml,242.593767,42.017380,1.524973e+07
1,2023-03-24 15:04:04.090,33.180167,-116.411667,11860.0,2.87,ml,355.136878,47.918134,1.501064e+07
2,2023-03-24 14:07:44.550,33.183167,-116.416000,11800.0,2.01,ml,355.585675,47.553815,1.500726e+07
3,2023-03-24 13:59:00.160,33.180333,-116.410833,12470.0,2.31,ml,355.042838,47.906377,1.500674e+07
4,2023-03-22 03:21:34.900,33.465333,-116.569833,9260.0,2.56,ml,33.363541,19.304096,1.479569e+07
...,...,...,...,...,...,...,...,...,...
160,2022-10-05 04:51:30.350,37.327833,-120.041167,19550.0,3.01,ml,140.666788,525.267022,2.858868e+05
161,2022-10-04 16:38:33.910,32.338167,-115.244333,2830.0,3.57,ml,321.590002,180.919294,2.419104e+05
162,2022-10-03 16:08:09.970,33.924667,-116.921500,7620.0,3.01,ml,128.765512,55.472148,1.536864e+05
163,2022-10-03 08:24:33.660,33.869667,-117.764667,4270.0,3.08,ml,102.958875,124.674213,1.258701e+05


## Plots

In [1]:
def __plot_timeline(catalog_df, catalog_all_df):

    fig, ax = plt.subplots(1, 1, figsize=(15,5))

    cax = ax.scatter(catalog_all_df.timestamp, catalog_all_df.distances_km, s=4**catalog_all_df.magnitude, 
                     c=catalog_all_df.magnitude, alpha=0.6, cmap='viridis')

#     ax.scatter(catalog_df.timestamp, catalog_df.distances_km, s=4**catalog_df.magnitude, 
#                facecolors='none', edgecolors='r', lw=1.0, zorder=-1)

    plt.colorbar(cax, ax=ax, pad=0.01)

    ax.set_ylim(bottom=0)
    ax.set_ylabel("Distance (km)", fontsize=14)

    ax.grid(ls=":", zorder=-1)
    
#     ax.set_xlim(events_all_df.timestamp.min().date(), events_all_df.timestamp.max().date())
#     ax.set_xlim(obs.UTCDateTime("2022-09-29"), obs.UTCDateTime("2023-04-03"))

    ax.set_title(f"triggered={len(catalog_df)}")
    
    plt.show();
    return fig

In [2]:
fig = __plot_timeline(events_df, events_all_df)

NameError: name 'events_df' is not defined